In [1]:
!pip install deap update_checker tqdm stopit xgboost
!pip install dask[delayed] dask[dataframe] dask-ml fsspec>=0.3.3 distributed>=2.10.0
!pip install scikit-mdr skrebate
!pip install tpot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11937 sha256=4915a58d356a870dbd83a619555155796895ae1868a1f566245606afe7205dcb
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit
  Preparing metadata (setup.py) ... done
  Created wheel for skrebate: filename=skrebate-0.62-py3-none-any.whl size=29253 sha256=fabaec25d816d5c70e391d46c1a80a11dd7324cce03c4f89989c8007ebfe3fe3
  Stored in directory: /root/.cache/pip/wheels/dd/67/40/683074a684607162bd0e34dcf7ccdfcab5861c3b2a83286f3a
Successfully built skrebate
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 2.1 MB/s eta 0:00:00


In [2]:
import torch
print(torch.__version__)

2.1.0+cu121


In [3]:
from tpot import TPOTClassifier
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

X, y = make_blobs(n_samples=100, centers=2, n_features=3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

clf = TPOTClassifier(config_dict='TPOT NN', template='Selector-Transformer-PytorchLRClassifier',
                     verbosity=2, population_size=10, generations=10)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
clf.export('tpot_nn_demo_pipeline.py')

Optimization Progress:   0%|          | 0/110 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 1.0

Generation 2 - Current best internal CV score: 1.0

Generation 3 - Current best internal CV score: 1.0

Generation 4 - Current best internal CV score: 1.0

Generation 5 - Current best internal CV score: 1.0

Generation 6 - Current best internal CV score: 1.0

Generation 7 - Current best internal CV score: 1.0

Generation 8 - Current best internal CV score: 1.0

Generation 9 - Current best internal CV score: 1.0

Generation 10 - Current best internal CV score: 1.0

Best pipeline: PytorchLRClassifier(MaxAbsScaler(SelectPercentile(input_matrix, percentile=57)), batch_size=4, learning_rate=1.0, num_epochs=15, weight_decay=0)
1.0


In [11]:
from deap import creator
from sklearn.model_selection import cross_val_score
import numpy as np

print(clf.score(X_test, y_test))
# print part of the pipeline dictionary
print(dict(list(clf.evaluated_individuals_.items())[0:2]))
# print a pipeline and its values
pipeline_str = list(clf.evaluated_individuals_.keys())[0]
print(pipeline_str)
print(clf.evaluated_individuals_[pipeline_str])
# convert pipeline string to scikit-learn pipeline object
optimized_pipeline = creator.Individual.from_string(pipeline_str, clf._pset)  # deap object
fitted_pipeline = clf._toolbox.compile(expr=optimized_pipeline)  # scikit-learn pipeline object
# print scikit-learn pipeline object
print(fitted_pipeline)

# Fix random state when the operator allows (optional) just for getting consistent CV score
for step in fitted_pipeline.steps:
    if 'random_state' in step[1].get_params():
        step[1].set_params(random_state=42)

# CV scores from scikit-learn
scores = cross_val_score(fitted_pipeline, X_train, y_train, cv=5, scoring='accuracy', verbose=0)
print(np.mean(scores))
print(clf.evaluated_individuals_[pipeline_str])


1.0
{'PytorchLRClassifier(MaxAbsScaler(SelectPercentile(input_matrix, SelectPercentile__percentile=57)), PytorchLRClassifier__batch_size=4, PytorchLRClassifier__learning_rate=1.0, PytorchLRClassifier__num_epochs=15, PytorchLRClassifier__weight_decay=0)': {'generation': 0, 'mutation_count': 0, 'crossover_count': 0, 'predecessor': ('ROOT',), 'operator_count': 3, 'internal_cv_score': 1.0}, 'PytorchLRClassifier(OneHotEncoder(SelectFwe(input_matrix, SelectFwe__alpha=0.039), OneHotEncoder__minimum_fraction=0.2, OneHotEncoder__sparse=False, OneHotEncoder__threshold=10), PytorchLRClassifier__batch_size=8, PytorchLRClassifier__learning_rate=0.01, PytorchLRClassifier__num_epochs=15, PytorchLRClassifier__weight_decay=0.001)': {'generation': 0, 'mutation_count': 0, 'crossover_count': 0, 'predecessor': ('ROOT',), 'operator_count': 3, 'internal_cv_score': 1.0}}
PytorchLRClassifier(MaxAbsScaler(SelectPercentile(input_matrix, SelectPercentile__percentile=57)), PytorchLRClassifier__batch_size=4, Pytorc